In [8]:
#Imports
from skimage import io
from skimage.color import rgb2grey
from skimage import exposure
from skimage.restoration import denoise_tv_chambolle
from _frangi import frangi, hessian
from skimage.filters import threshold_li
from skimage import morphology
from skimage.morphology import skeletonize
import warnings as war
%matplotlib notebook
import matplotlib.pyplot as plt
import numpy as np
from os import walk
from os import path
import csv

#Importar imágenes
def csvReader(csvPath):
    rows = []
    with open(csvPath, newline='') as csvfile:
        perfiles = csv.reader(csvfile, delimiter=',', quoting=csv.QUOTE_NONNUMERIC)
        for row in perfiles:
            rows.append(row)
    return rows  

In [ ]:
# Importar imágenes
fullImage = 'Optimize\\Full_Image.png'
fullMask = 'Optimize\\Full_Mask.png'
fullCsv = csvReader('Optimize\\Full_Csv.csv')

In [7]:
# Funcion sobre la que modificar parametros
def processImage(img, clipLimit, nb, w, removeSamll, conn):
    """
    Apply some filters to the given image.
    :param img: image to process
    :param clipLimit: clip_limit parameter (0,1)
    :param nb: nbins parameter (0,100)
    :param w: weight parameter (0,1)
    :param removeSmall: min_size parameter (0,100)
    :param conn: connectivity parameter (0,100)
    :return: processed image
    """
    result = None
    # disable warnings
    with war.catch_warnings():
        war.simplefilter("ignore")
        # Convert image to grey scale
        img = rgb2grey(img)
        # Equalize histogram
        imgAdapted = exposure.equalize_adapthist(img, clip_limit=clipLimit, nbins=nb)
        # Reduce noise
        imgAdaptedDenoise = denoise_tv_chambolle(imgAdapted, weight=w)
        # Apply frangi filter
        frangiImg = frangi(imgAdaptedDenoise)
        # Calculate image threshold
        threshold = threshold_li(frangiImg)
        # Apply threshold
        thresholdedImg = frangiImg >= threshold
        # Skeletonize image
        skImg = skeletonize(thresholdedImg)
        # Remove small objects (min_size fixed to 70)
        result = morphology.remove_small_objects(skImg, min_size=removeSmall, connectivity=conn)
    return result

In [1]:
#Funcion a optimizar. From wiki.
def distance(str1, str2):
    """
    Distancia de Levenshtein
    """
    d=dict()
    for i in range(len(str1)+1):
        d[i]=dict()
        d[i][0]=i
    for i in range(len(str2)+1):
        d[0][i] = i
    for i in range(1, len(str1)+1):
        for j in range(1, len(str2)+1):
            d[i][j] = min(d[i][j-1]+1, d[i-1][j]+1, d[i-1][j-1]+(not str1[i-1] == str2[j-1]))
    return d[len(str1)][len(str2)]

In [ ]:
#Optimizar
from scipy import optimize

